# Description

The notebook contains full versions of DAG output checks that compare the prod system output vs that of the prod simulation run.

# Imports

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import logging

import core.config as cconfig
import dataflow.core as dtfcore
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hpandas as hpandas
import helpers.hprint as hprint
import reconciliation as reconcil

/app/amp/dataflow/core/dag.py:15: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.9/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-b8910937-4719-43f7-918b-b39d4dd48d76.json'
INFO  # Git
  branch_name='CmTask6964_Add_only_warning_to_get_system_log_paths'
  hash='9c6e4b652'
  # Last commits:
    *   9c6e4b652 Sameep Pote Merge branch 'master' into CmTask6964_Add_only_warning_to_get_system_log_paths ( 9 minutes ago) Thu Feb 1 22:20:03 2024  (HEAD -> CmTask6964_Add_only_warning_to_get_system_log_paths, origin/CmTask6964_Add_only_warning_to_get_system_log_paths)
    |\  
    * | d02a60629 Sameep2808 nits                                                              (10 minutes ago) Thu Feb 1 22:19:39 2024           
    * | b8806f01b Sameep2808 Fixing test cases                                                 (17 minutes ago) Thu Feb 1 22:12:02 2024           
# Machine info
  system=Linux
  node name=ca8e7dd1b33b
  release=5.15.0-1051-aws
  version=#56~20.04.1-Ubuntu SMP Tue Nov 28 15:43:31 UTC 2023
  machi

# Build the reconciliation config

In [4]:
# When running manually, specify the path to the config to load config from file,
# for e.g., `.../reconciliation_notebook/fast/result_0/config.pkl`.
config_file_name = None
# Set 'replace_ecs_tokyo = True' if running the notebook manually.
replace_ecs_tokyo = False
config = cconfig.get_notebook_config(
    config_file_path=config_file_name, replace_ecs_tokyo=replace_ecs_tokyo
)
if config is None:
    _LOG.info("Using hardwired config")
    # Specify the config directly when running the notebook manually.
    # Below is just an example.
    dst_root_dir = "/shared_data/ecs/preprod/prod_reconciliation/"
    dag_builder_ctor_as_str = (
        "dataflow_orange.pipelines.C3.C3a_pipeline_tmp.C3a_DagBuilder_tmp"
    )
    start_timestamp_as_str = "20231103_131000"
    end_timestamp_as_str = "20231104_130500"
    run_mode = "paper_trading"
    mode = "scheduled"
    check_dag_output_self_consistency = True
    config_list = reconcil.build_reconciliation_configs(
        dst_root_dir,
        dag_builder_ctor_as_str,
        start_timestamp_as_str,
        end_timestamp_as_str,
        run_mode,
        mode,
        check_dag_output_self_consistency=check_dag_output_self_consistency,
    )
    config = config_list[0]
print(config)

INFO  Using hardwired config
INFO  run_date='20240112'
INFO  Using run_date=20240112
INFO  dst_root_dir='/shared_data/ecs/preprod/prod_reconciliation/', dag_builder_name='C11a', run_mode='paper_trading', start_timestamp_as_str='20240112_133000', end_timestamp_as_str='20240113_130000'
INFO  target_dir='/shared_data/ecs/preprod/prod_reconciliation/C11a/paper_trading/20240112_133000.20240113_130000'
INFO  system_log_dir='system_log_dir.scheduled'
INFO  system_log_dir='system_log_dir.scheduled'
meta: 
  date_str: 20240112
  asset_class: crypto
  run_tca: False
  bar_duration: 30T
  compute_research_portfolio_mode: automatic
  compute_dag_memory_stats: automatic
system_log_path: 
  prod: /shared_data/ecs/preprod/prod_reconciliation/C11a/paper_trading/20240112_133000.20240113_130000/prod/system_log_dir.scheduled
  sim: /shared_data/ecs/preprod/prod_reconciliation/C11a/paper_trading/20240112_133000.20240113_130000/simulation/system_log_dir.scheduled
dag_builder_ctor_as_str: dataflow_lemonade.

# Specify data to load

In [5]:
# The dict points to `system_log_dir` for different experiments.
system_log_path_dict = dict(config["system_log_path"].to_dict())

In [6]:
# This dict points to `system_log_dir/dag/node_io/node_io.data` for different experiments.
data_type = "dag_data"
dag_path_dict = reconcil.get_system_log_paths(
    system_log_path_dict, data_type, only_warning=True
)
dag_path_dict

{'prod': '/shared_data/ecs/preprod/prod_reconciliation/C11a/paper_trading/20240112_133000.20240113_130000/prod/system_log_dir.scheduled/dag/node_io/node_io.data',
 'sim': '/shared_data/ecs/preprod/prod_reconciliation/C11a/paper_trading/20240112_133000.20240113_130000/simulation/system_log_dir.scheduled/dag/node_io/node_io.data'}

# DAG io

## Load

In [7]:
# Get DAG node names.
get_dag_output_mode = config["meta"]["get_dag_output_mode"]
dag_path = reconcil.get_dag_output_path(dag_path_dict, get_dag_output_mode)
dag_node_names = dtfcore.get_dag_node_names(dag_path)
_LOG.info(
    "First node='%s' / Last node='%s'", dag_node_names[0], dag_node_names[-1]
)

INFO  First node='predict.0.read_data' / Last node='predict.10.process_forecasts'


In [8]:
# Get timestamps for the last DAG node.
dag_node_timestamps = dtfcore.get_dag_node_timestamps(
    dag_path, dag_node_names[-1], as_timestamp=True
)
_LOG.info(
    "First timestamp='%s'/ Last timestamp='%s'",
    dag_node_timestamps[0][0],
    dag_node_timestamps[-1][0],
)

INFO  First timestamp='2024-01-12 08:30:00-05:00'/ Last timestamp='2024-01-13 08:00:00-05:00'


In [9]:
# Get DAG output for the last node and the last timestamp.
dag_df = dtfcore.load_dag_outputs(dag_path, dag_node_names[-1])
_LOG.info("Output of the last node:\n")
hpandas.df_to_str(dag_df, num_rows=5, log_level=logging.INFO)

INFO  Output of the last node:



feature                                                                                                                                                                                                                                                              p1.c.xs_adj.c                                                                                                                                                                                                                                                              p2.c.xs_adj.c                                                                                                                                                                                                                                                              p_lr.c.xs_adj.c                                                                                                                                                                                                                                                              p_diff.c.xs_adj.c                                                                                                                                                                                                                                                              q1.c.xs_adj.c                                                                                                                                                                                                                                                              q2.c.xs_adj.c                                                                                                                                                                                                                                                              q3.c.xs_adj.c                                                                                                                                                                                                                                                              r1.c.xs_adj.c                                                                                                                                                                                                                                                              r2.c.xs_adj.c                                                                                                                                                                                                                                                              r4.c.xs_adj.c                                                                                                                                                                                                                                                              r5.c.xs_adj.c                                                                                                                                                                                                                                                              r6.c.xs_adj.c                                                                                                                                                                                                                                                              r_lr.c.xs_adj.c                                                                                                                                                                                                                                                              r_est.c.xs_adj.c                                                                                                                                                                                                                                                    

## Check DAG io self-consistency

In [10]:
# TODO(Grisha): pass the value via config.
diff_threshold = 1e-3
compare_dfs_kwargs = {
    # TODO(Nina): CmTask4387 "DAG self-consistency check fails when
    # `history_lookback=15T` for C3a".
    "compare_nans": False,
    "diff_mode": "pct_change",
    "assert_diff_threshold": None,
}

In [11]:
# Run for all timestamps.
bar_timestamp = "all"
if config["check_dag_output_self_consistency"]:
    # Self-consistency check fails for C11a. See CmTask6519
    # for more details. Also for C12a, see CmTask7053.
    # Compare DAG output at T with itself at time T-1.
    dtfcore.check_dag_output_self_consistency(
        dag_path,
        dag_node_names[-1],
        bar_timestamp,
        trading_freq=config["meta"]["bar_duration"],
        diff_threshold=diff_threshold,
        **compare_dfs_kwargs,
    )

## Compare DAG io (prod vs sim)

In [12]:
# Run for all nodes and all timestamps.
bar_timestamp = "all"
node_name = "all"
_ = dtfcore.compare_dag_outputs(
    dag_path_dict,
    node_name,
    bar_timestamp,
    diff_threshold=diff_threshold,
    **compare_dfs_kwargs,
)